# word2vec, fastTextを用いた日本語単語のベクトル表現の実装

## 目標
1.	学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
2.	学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる

In [ ]:
# 単語分割にMeCab+NEologdを使用
import MeCab

m_t = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
